### **Introduction to the Smartwatch Product Recommendation Chatbot**

This code implements a **smartwatch product recommendation chatbot** powered by 
- **large language model (LLM)**, 
- **Singe Agent with Prompt (via Langchain)**, and 
- **Retrieval-Augmented Generation (RAG)** techniques. 

The system is designed to 
recommend smartwatches based on user preferences and product features, while ensuring the response 
appears natural and not explicitly tied to the provided context.

### Key Components and Workflow:
1. **LLM Integration**:  
   The chatbot uses the **Qwen3:14B** model via the `langchain_ollama` library, which is hosted on a 
local LLM server. This model is responsible for understanding user queries, retrieving relevant 
product information, and generating tailored recommendations.

2. **Product Catalog**:  
   A static product catalog is embedded in the code, containing details about two smartwatches:  
   - **Military-grade "ABC123"**: Waterproof (IP68), long battery life (20+ days), heavy (300g).  
   - **Leisure-grade "DEF456"**: Lightweight (50g), non-waterproof, medium battery life (~50 hours).  


3. **RAG Pipeline**:  
   - The product catalog is converted into a **vector store** using `InMemoryVectorStore`, enabling 
efficient semantic search.  
   - When a user asks a question (e.g., "Recommend a product for underwater outdoor activities"), the 
system retrieves the most relevant product details from the vector store.  

4. **Prompt Engineering**:  
   - A **system prompt** guides the LLM to answer questions based on the retrieved context without 
explicitly mentioning the source.  
   - A **chain** is constructed using `ChatPromptTemplate`, combining the retrieved product data, 
user question, and instructions for the LLM.  

5. **Output**:  
   The final recommendation is generated by the LLM and returned as a natural language response. For 
example, if a user asks for a waterproof smartwatch for underwater activities, the system would 
prioritize recommending the **military-grade "ABC123"** model.  

### Example Use Case:  
If a user says:  
> "Recommend me a product for me. I am a businessman and love outdoor activities under water."  

The chatbot will:  
1. Retrieve the waterproof "ABC123" product details from the vector store.  
2. Generate a recommendation highlighting its IP68 rating and long battery life, tailored to the 
user’s needs.  

This approach ensures the chatbot provides accurate, context-aware recommendations while maintaining 
a seamless user experience.


## Code impelementation

In [9]:

from langchain_ollama import ChatOllama
from langchain_ollama import OllamaEmbeddings
from langchain_ollama.llms import Client
from langchain_core.vectorstores import InMemoryVectorStore
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
import os
from dotenv import load_dotenv
load_dotenv()

True

In [10]:
#   setup LLM server
llm_server_url    = os.getenv('LLM_SERVER_URL')
llm_model_name    = "qwen3:14b"
client = Client(llm_server_url)
llm = ChatOllama(_client=client, model=llm_model_name)
embeddings = OllamaEmbeddings(model=llm_model_name)

In [11]:
#   smartwatch product catalog 
context = """
Below are our smart watch products, including product features and introduction::
    - Product Code: "ABC123"
        - product grade: military
        - product features: 
            - waterproof (IP68)
            - long battery life (support over 20 days)
            - weight is heavy (300g)
            
    - Product Code: "DEF456
        - product grade: leisure
        - product features: 
            - weight is light (50g)
            - non-waterproof
            - medium battery life (~ 50 hours)
"""

In [12]:
#   RAG (convert smartwatch product catalog to vectorstore)
vectorstore = InMemoryVectorStore.from_texts([context], embedding=embeddings)
retriever = vectorstore.as_retriever()

In [13]:
#   write single agent & prompt
system_message = """
Answer the following question based on this context but Do not mention the information is provided by context:
Question: {human_question}

Based on the above context:
When you recommend a product based on the `product_catalog_context`, you must think step by step.
"""

product_catalog = """
    Below are product catalog: 
        <product_catalog_context>
        {product_catalog_context}
        </product_catalog_context>
"""

In [14]:
#   setup chain
chain = ChatPromptTemplate.from_messages([
    ("system", system_message),
    ("system", product_catalog),
    ("human", "{human_question}")
])

Product_Recommendation_QA_chain = (
        {
            "product_catalog_context": retriever,
            "human_question": RunnablePassthrough(),
        } | chain | llm | StrOutputParser()
)

### chatbot service

In [8]:
#   chatbot session start
human_question = "Recommend me a product for me. I am a business man, and love outdoor activities under water. List out all for me"
ai_product_recommendation = Product_Recommendation_QA_chain.invoke(input=human_question)
print(ai_product_recommendation)


<think>
Okay, the user is a businessman who loves underwater outdoor activities. Let me check the product catalog.

First, there are two smartwatches: ABC123 and DEF456. 

For ABC123: It's military-grade, waterproof (IP68), battery lasts over 20 days, but it's heavy (300g). Since the user does underwater activities, waterproof is essential. The long battery life is good for outdoor trips. However, the weight might be a downside if he prefers lightweight gear.

DEF456 is leisure-grade, not waterproof, light (50g), but battery only lasts 50 hours. Since the user needs waterproofing for underwater activities, DEF456 isn't suitable. 

So, only ABC123 meets the waterproof requirement. Even though it's heavy, the other features might be worth it. I should recommend ABC123 and mention the trade-offs.
</think>

Based on your needs as a businessman who enjoys underwater outdoor activities, here’s a tailored recommendation:

### **Recommended Product:**
**Product Code: "ABC123"**  
- **Grade:** 